 ## Installation and Setup

In [1]:
# Install required packages
!pip install flask flask-cors requests termcolor pyngrok bitsandbytes -q
!pip install -U langchain langchain-openai -q

print("All packages installed successfully!")
print("Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)")

All packages installed successfully!
Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/genai_hw_MCP_A2A

/content/drive/MyDrive/genai_hw_MCP_A2A


In [4]:
import json
from flask import Flask, request, Response, jsonify
from flask_cors import CORS
import sqlite3
import threading
import time
import requests
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AnyMessage
from langgraph.graph import StateGraph, START, END

from database_setup import DatabaseSetup

## Database Setup

In [5]:
def init_db(db_path: str = "support.db") -> sqlite3.Connection:
    """
    Initialize the SQLite database using DatabaseSetup
    and return a shared connection.
    """
    # Run setup (create tables, sample data, etc.)
    setup = DatabaseSetup(db_path=db_path)
    setup.connect()
    setup.create_tables()
    setup.create_triggers()
    setup.insert_sample_data()
    setup.close()

    conn = sqlite3.connect(db_path, check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn

db_conn = init_db()

Connected to database: support.db
Tables created successfully!
Triggers created successfully!
Sample data inserted successfully!
  - 15 customers added
  - 25 tickets added
Database connection closed.


## Customer Management Functions

In [6]:
def get_customer(args: Dict[str, Any]) -> Dict[str, Any]:
    if isinstance(args, int):
        args = {"customer_id": args}

    customer_id = args.get("customer_id")
    if customer_id is None:
        return {"success": False, "error": "customer_id is required"}

    cur = db_conn.cursor()
    cur.execute("SELECT * FROM customers WHERE id = ?", (customer_id,))
    row = cur.fetchone()

    if not row:
        return {"success": False, "error": f"No customer found with id {customer_id}"}

    return {
        "success": True,
        "data": dict(row),
    }


def list_customers(args: Dict[str, Any]) -> Dict[str, Any]:
    status = args.get("status")   # e.g. 'active' or 'disabled'
    limit = args.get("limit", 10)

    query = "SELECT * FROM customers"
    params: List[Any] = []

    if status:
        query += " WHERE status = ?"
        params.append(status)

    query += " ORDER BY id LIMIT ?"
    params.append(limit)

    cur = db_conn.cursor()
    cur.execute(query, tuple(params))
    rows = cur.fetchall()

    return {
        "success": True,
        "count": len(rows),
        "customers": [dict(r) for r in rows],
    }


def update_customer(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Args should look like:
    {
      "customer_id": 5,
      "data": {
        "email": "new@email.com",
        "phone": "+1-555-0123",
        ...
      }
    }
    """
    customer_id = args.get("customer_id")
    data = args.get("data", {})

    if not customer_id:
        return {"success": False, "error": "customer_id is required"}
    if not data:
        return {"success": False, "error": "data dict is required"}

    # Only allow certain fields
    allowed_fields = {"name", "email", "phone", "status"}
    fields = [k for k in data.keys() if k in allowed_fields]

    if not fields:
        return {"success": False, "error": "No valid fields to update"}

    set_clause = ", ".join(f"{f} = ?" for f in fields)
    params = [data[f] for f in fields]
    params.append(customer_id)

    cur = db_conn.cursor()
    cur.execute(f"UPDATE customers SET {set_clause} WHERE id = ?", tuple(params))
    db_conn.commit()

    if cur.rowcount == 0:
        return {"success": False, "error": f"No customer found with id {customer_id}"}

    # Return updated customer
    return get_customer({"customer_id": customer_id})


def create_ticket(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Args:
    {
      "customer_id": 5,
      "issue": "Double charged",
      "priority": "high"
    }
    """
    customer_id = args.get("customer_id")
    issue = args.get("issue")
    priority = args.get("priority", "medium")

    if not customer_id:
        return {"success": False, "error": "customer_id is required"}
    if not issue:
        return {"success": False, "error": "issue is required"}

    # Validate priority
    if priority not in ("low", "medium", "high"):
        return {"success": False, "error": "priority must be one of: low, medium, high"}

    cur = db_conn.cursor()

    # Ensure customer exists
    cur.execute("SELECT 1 FROM customers WHERE id = ?", (customer_id,))
    if not cur.fetchone():
        return {"success": False, "error": f"No customer found with id {customer_id}"}

    cur.execute(
        """
        INSERT INTO tickets (customer_id, issue, status, priority)
        VALUES (?, ?, 'open', ?)
        """,
        (customer_id, issue, priority),
    )
    db_conn.commit()

    ticket_id = cur.lastrowid

    cur.execute("SELECT * FROM tickets WHERE id = ?", (ticket_id,))
    ticket_row = cur.fetchone()

    return {
        "success": True,
        "ticket": dict(ticket_row),
    }


def get_customer_history(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Args:
    {
      "customer_id": 5
    }
    """
    customer_id = args.get("customer_id")
    if not customer_id:
        return {"success": False, "error": "customer_id is required"}

    cur = db_conn.cursor()
    cur.execute(
        """
        SELECT * FROM tickets
        WHERE customer_id = ?
        ORDER BY created_at DESC
        """,
        (customer_id,),
    )
    rows = cur.fetchall()

    return {
        "success": True,
        "count": len(rows),
        "tickets": [dict(r) for r in rows],
    }

In [7]:
# Test the functions
print("Customer management functions defined successfully!")
print("\n Available functions:")
print("   - get_customer(customer_id)")
print("   - list_customers(status, limit)")
print("   - update_customer(customer_id, data)")
print("   - create_ticket(customer_id, issue, priority)")
print("   - get_customer_history(customer_id)")

# Quick test
print("\n Quick test - Fetching customer ID 1:")
result = get_customer(1)
if result['success']:
    customer = result['data']
    print(f"   Name: {customer['name']}")
    print(f"   Email: {customer['email']}")
    print(f"   Status: {customer['status']}")

# Test list_customers
print("\n TEST – list_customers (limit 3):")
result = list_customers({"limit": 3})
if result["success"]:
    for c in result["customers"]:
        print(f" - {c['id']} | {c['name']} | {c['status']}")
else:
    print(result["error"])

# Test update_customer
print("\n TEST – update_customer (change email & phone for ID 1):")
result = update_customer({
    "customer_id": 1,
    "data": {
        "email": "john.new@example.com",
        "phone": "+1-555-7777"
    }
})
print(result)

# Test create_ticket
print("\n TEST – create_ticket for customer 1:")
result = create_ticket({
    "customer_id": 1,
    "issue": "Unable to login",
    "priority": "high"
})
print(result)

# Test get_customer_history
print("\n TEST – get_customer_history for customer 1:")
result = get_customer_history({"customer_id": 1})
if result["success"]:
    for t in result["tickets"]:
        print(f" - Ticket #{t['id']} | {t['issue']} | {t['status']} | {t['priority']}")
else:
    print(result["error"])

Customer management functions defined successfully!

 Available functions:
   - get_customer(customer_id)
   - list_customers(status, limit)
   - update_customer(customer_id, data)
   - create_ticket(customer_id, issue, priority)
   - get_customer_history(customer_id)

 Quick test - Fetching customer ID 1:
   Name: John Doe
   Email: john.new@example.com
   Status: active

 TEST – list_customers (limit 3):
 - 1 | John Doe | active
 - 2 | Jane Smith | active
 - 3 | Bob Johnson | disabled

 TEST – update_customer (change email & phone for ID 1):
{'success': True, 'data': {'id': 1, 'name': 'John Doe', 'email': 'john.new@example.com', 'phone': '+1-555-7777', 'status': 'active', 'created_at': '2025-12-01 23:18:40', 'updated_at': '2025-12-01 23:40:10'}}

 TEST – create_ticket for customer 1:
{'success': True, 'ticket': {'id': 81, 'customer_id': 1, 'issue': 'Unable to login', 'status': 'open', 'priority': 'high', 'created_at': '2025-12-01 23:40:10'}}

 TEST – get_customer_history for customer

## MCP HTTP Streaming Server Implementation

In [8]:
# ================================
# MCP Server
# ================================

# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Server state
server_thread = None
server_running = False

# =====================
# MCP Tool Definitions
# =====================

MCP_TOOLS = [
    {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, and status.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to retrieve"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "list_customers",
        "description": "List customers in the database. Can optionally filter by status and limit the number of results.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "Optional filter by customer status"
                },
                "limit": {
                    "type": "integer",
                    "description": "Maximum number of customers to return (default 10)"
                }
            }
        }
    },
    {
        "name": "update_customer",
        "description": "Update an existing customer's information. Provide the customer ID and any fields you want to change.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to update"
                },
                "name": {
                    "type": "string",
                    "description": "New name (optional)"
                },
                "email": {
                    "type": "string",
                    "description": "New email (optional)"
                },
                "phone": {
                    "type": "string",
                    "description": "New phone (optional)"
                },
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "New status (optional)"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "create_ticket",
        "description": "Create a support ticket for a specific customer.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "ID of the customer who opened the ticket"
                },
                "issue": {
                    "type": "string",
                    "description": "Description of the customer's issue"
                },
                "priority": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Ticket priority (default: medium)"
                }
            },
            "required": ["customer_id", "issue"]
        }
    },
    {
        "name": "get_customer_history",
            "description": "Get all tickets associated with a given customer, ordered by most recent first.",
            "inputSchema": {
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "integer",
                        "description": "ID of the customer whose ticket history to fetch"
                    }
                },
                "required": ["customer_id"]
            }
    }
]

# ==========================
# SSE helper
# ==========================

def create_sse_message(data: Dict[str, Any]) -> str:
    """
    Format a message for Server-Sent Events (SSE).
    SSE format: 'data: {json}\\n\\n'
    """
    return f"data: {json.dumps(data)}\n\n"

# ==================================
# MCP handlers (initialize / list)
# ==================================

def handle_initialize(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle MCP initialize request.
    This is the first message in the MCP protocol handshake.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "protocolVersion": "2024-11-05",
            "capabilities": {
                "tools": {},
            },
            "serverInfo": {
                "name": "customer-support-mcp-server",
                "version": "1.0.0"
            }
        }
    }

def handle_tools_list(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/list request.
    Returns the list of available tools.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "tools": MCP_TOOLS
        }
    }

# ==================================
# Tool adapters
# ==================================

def mcp_get_customer(customer_id: int) -> Dict[str, Any]:
    """Adapter for get_customer(args)."""
    return get_customer({"customer_id": customer_id})

def mcp_list_customers(status: str = None, limit: int = 10) -> Dict[str, Any]:
    """Adapter for list_customers(args)."""
    args: Dict[str, Any] = {"limit": limit}
    if status:
        args["status"] = status
    return list_customers(args)

def mcp_update_customer(
    customer_id: int,
    name: str = None,
    email: str = None,
    phone: str = None,
    status: str = None,
) -> Dict[str, Any]:
    """Adapter for update_customer(args) """
    data: Dict[str, Any] = {}
    if name is not None:
        data["name"] = name
    if email is not None:
        data["email"] = email
    if phone is not None:
        data["phone"] = phone
    if status is not None:
        data["status"] = status

    return update_customer({
        "customer_id": customer_id,
        "data": data,
    })

def mcp_create_ticket(customer_id: int, issue: str, priority: str = "medium") -> Dict[str, Any]:
    """Adapter for create_ticket(args)."""
    return create_ticket({
        "customer_id": customer_id,
        "issue": issue,
        "priority": priority,
    })

def mcp_get_customer_history(customer_id: int) -> Dict[str, Any]:
    """Adapter for get_customer_history(args)."""
    return get_customer_history({"customer_id": customer_id})

# ==================================
# tools/call handler
# ==================================

def handle_tools_call(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/call request.
    Executes the requested tool and returns the result.
    """
    params = message.get("params", {})
    tool_name = params.get("name")
    arguments = params.get("arguments", {})

    # Map tool names to adapter functions
    tool_functions = {
        "get_customer": mcp_get_customer,
        "list_customers": mcp_list_customers,
        "update_customer": mcp_update_customer,
        "create_ticket": mcp_create_ticket,
        "get_customer_history": mcp_get_customer_history,
    }

    if tool_name not in tool_functions:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Tool not found: {tool_name}"
            }
        }

    try:
        result = tool_functions[tool_name](**arguments)

        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "result": {
                "content": [
                    {
                        "type": "text",
                        "text": json.dumps(result, indent=2)
                    }
                ]
            }
        }
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32603,
                "message": f"Tool execution error: {str(e)}"
            }
        }

# ===================
# MCP message router
# ===================

def process_mcp_message(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Process an MCP message and route it to the appropriate handler.
    """
    method = message.get("method")

    if method == "initialize":
        return handle_initialize(message)
    elif method == "tools/list":
        return handle_tools_list(message)
    elif method == "tools/call":
        return handle_tools_call(message)
    else:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Method not found: {method}"
            }
        }

# =====================
# Flask Routes (SSE)
# =====================

@app.route('/mcp', methods=['POST'])
def mcp_endpoint():
    """
    Main MCP endpoint for MCP communication.
    Receives MCP messages and streams responses using Server-Sent Events.
    """
    message = request.get_json()

    def generate():
        try:
            print(f"[INFO] Received MCP message: {message.get('method')}")
            response = process_mcp_message(message)
            print("[INFO] Sending MCP response")
            yield create_sse_message(response)
        except Exception as e:
            error_response = {
                "jsonrpc": "2.0",
                "id": message.get("id"),
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {str(e)}"
                }
            }
            yield create_sse_message(error_response)

    return Response(generate(), mimetype='text/event-stream')

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint to verify server is running."""
    return jsonify({
        "status": "healthy",
        "server": "customer-support-mcp-server",
        "version": "1.0.0"
    })

print("[SUCCESS] MCP Server implementation complete!")
print(f"   - {len(MCP_TOOLS)} tools exposed:")
for t in MCP_TOOLS:
    print("     •", t["name"])


[SUCCESS] MCP Server implementation complete!
   - 5 tools exposed:
     • get_customer
     • list_customers
     • update_customer
     • create_ticket
     • get_customer_history


## Start the MCP Server

In [9]:
HOST = "127.0.0.1"
PORT = 8000
SERVER_URL = f"http://{HOST}:{PORT}"

def run_server():
    """Run Flask server (blocking). Should be used inside a thread."""
    app.run(host=HOST, port=PORT, debug=False, use_reloader=False)

def start_server():
    """Start the MCP server in a background thread and wait for /health OK."""
    global server_thread, server_running

    if server_running:
        print("[INFO] Server is already running.")
        return

    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # Wait for health check to be ready
    print("[INFO] Starting server...")
    for i in range(30):
        try:
            r = requests.get(f"{SERVER_URL}/health", timeout=1)
            if r.status_code == 200:
                server_running = True
                print(f"[INFO] Server is up at {SERVER_URL}")
                print(f"   MCP endpoint: {SERVER_URL}/mcp")
                return
        except Exception:
            time.sleep(0.5)

    print("[ERROR] Failed to detect running server via /health")

def check_server_status():
    """Quick /health probe."""
    try:
        r = requests.get(f"{SERVER_URL}/health", timeout=2)
        print("Health:", r.status_code, r.json())
    except Exception as e:
        print("[ERROR] Health check failed:", e)

In [10]:
start_server()
check_server_status()

[INFO] Starting server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "GET /health HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "GET /health HTTP/1.1" 200 -


[INFO] Server is up at http://127.0.0.1:8000
   MCP endpoint: http://127.0.0.1:8000/mcp
Health: 200 {'server': 'customer-support-mcp-server', 'status': 'healthy', 'version': '1.0.0'}


In [11]:
def send_mcp_message(method: str, params: Dict[str, Any] = None, message_id: int = 1):
    """
    Send a single MCP message to /mcp and print/return the response.
    Uses SSE (Server-Sent Events)
    """
    if params is None:
        params = {}

    message = {
        "jsonrpc": "2.0",
        "id": message_id,
        "method": method,
        "params": params,
    }

    print("\n==============================")
    print(f"[INFO] MCP request → {method}")
    print(json.dumps(message, indent=2))

    resp = requests.post(
        f"{SERVER_URL}/mcp",
        json=message,
        stream=True,
        timeout=30,
    )

    if resp.status_code != 200:
        print("[ERROR] HTTP error:", resp.status_code, resp.text)
        return None

    full_payload = None

    print("[INFO] MCP response:")
    for line in resp.iter_lines(decode_unicode=True):
        if not line:
            continue
        if line.startswith("data: "):
            data_str = line[len("data: "):]
            try:
                payload = json.loads(data_str)
                full_payload = payload
                print(json.dumps(payload, indent=2))
            except Exception as e:
                print("[ERROR] Failed to parse JSON from SSE line:", e, "\nRaw:", data_str)

    return full_payload

In [12]:
class MCPToolClient:
    """
    Host-side client that talks to your MCP server via /mcp.
    Provides a simple call_tool(name, arguments) interface.
    """

    def __init__(self, server_url: str):
        self.server_url = server_url
        self._next_id = 1

    def _send_mcp_message(self, method: str, params: Dict[str, Any]) -> Dict[str, Any]:
        message_id = self._next_id
        self._next_id += 1

        message = {
            "jsonrpc": "2.0",
            "id": message_id,
            "method": method,
            "params": params,
        }

        resp = requests.post(
            f"{self.server_url}/mcp",
            json=message,
            stream=True,
            timeout=30,
        )

        if resp.status_code != 200:
            raise RuntimeError(f"MCP HTTP error: {resp.status_code} {resp.text}")

        full_payload = None
        for line in resp.iter_lines(decode_unicode=True):
            if not line:
                continue
            if line.startswith("data: "):
                data_str = line[len("data: "):]
                full_payload = json.loads(data_str)
                break

        if full_payload is None:
            raise RuntimeError("No SSE data received from MCP server")

        # If it's an error, raise
        if "error" in full_payload:
            raise RuntimeError(f"MCP error: {full_payload['error']}")

        return full_payload

    def call_tool(self, name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Call an MCP tool and return the parsed JSON result produced
        by your tool function (get_customer / list_customers / etc.).
        """
        payload = self._send_mcp_message(
            "tools/call",
            {
                "name": name,
                "arguments": arguments,
            },
        )

        # packed tool result as text JSON in handle_tools_call
        content_items = payload.get("result", {}).get("content", [])
        if not content_items:
            return {}

        text_json = content_items[0].get("text", "{}")
        return json.loads(text_json)

## Test Section: MCP Protocol in Action

### TEST 1 – initialize

In [13]:
init_result = send_mcp_message(
    "initialize",
    {
        "protocolVersion": "2024-11-05",
        "capabilities": {}
    },
    message_id=1
)

INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → initialize
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "capabilities": {}
  }
}
[INFO] Received MCP message: initialize
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "protocolVersion": "2024-11-05",
    "capabilities": {
      "tools": {}
    },
    "serverInfo": {
      "name": "customer-support-mcp-server",
      "version": "1.0.0"
    }
  }
}


### TEST 2 – tools/list

In [14]:
tools_result = send_mcp_message(
    "tools/list",
    {},
    message_id=2
)

INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/list
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/list",
  "params": {}
}
[INFO] Received MCP message: tools/list
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "tools": [
      {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, and status.",
        "inputSchema": {
          "type": "object",
          "properties": {
            "customer_id": {
              "type": "integer",
              "description": "The unique ID of the customer to retrieve"
            }
          },
          "required": [
            "customer_id"
          ]
        }
      },
      {
        "name": "list_customers",
        "description": "List customers in the database. Can optionally filter by status and limit the number of results.",
        "inputSchema": {
          "type": "object",
          "properties": {

### TEST 3 – tools/call

In [15]:
get_cust_raw = send_mcp_message(
    "tools/call",
    {
        "name": "get_customer",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=3
)

if get_cust_raw and "result" in get_cust_raw:
    text_payload = get_cust_raw["result"]["content"][0]["text"]
    get_cust_data = json.loads(text_payload)
    print("\n Parsed get_customer result:")
    print(json.dumps(get_cust_data, indent=2))


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "get_customer",
    "arguments": {
      "customer_id": 1
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 3,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"data\": {\n    \"id\": 1,\n    \"name\": \"John Doe\",\n    \"email\": \"john.new@example.com\",\n    \"phone\": \"+1-555-7777\",\n    \"status\": \"active\",\n    \"created_at\": \"2025-12-01 23:18:40\",\n    \"updated_at\": \"2025-12-01 23:40:10\"\n  }\n}"
      }
    ]
  }
}

 Parsed get_customer result:
{
  "success": true,
  "data": {
    "id": 1,
    "name": "John Doe",
    "email": "john.new@example.com",
    "phone": "+1-555-7777",
    "status": "active",
    "created_at": "2025-12-01 23:18:40",
    "updated_at": "2025-12-01 23:40:10"
  }
}


In [16]:
list_raw = send_mcp_message(
    "tools/call",
    {
        "name": "list_customers",
        "arguments": {
            "status": "active",
            "limit": 5
        }
    },
    message_id=4
)

if list_raw and "result" in list_raw:
    text_payload = list_raw["result"]["content"][0]["text"]
    list_data = json.loads(text_payload)
    print("\n Parsed list_customers result:")
    print(json.dumps(list_data, indent=2))


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 4,
  "method": "tools/call",
  "params": {
    "name": "list_customers",
    "arguments": {
      "status": "active",
      "limit": 5
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 4,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"count\": 5,\n  \"customers\": [\n    {\n      \"id\": 1,\n      \"name\": \"John Doe\",\n      \"email\": \"john.new@example.com\",\n      \"phone\": \"+1-555-7777\",\n      \"status\": \"active\",\n      \"created_at\": \"2025-12-01 23:18:40\",\n      \"updated_at\": \"2025-12-01 23:40:10\"\n    },\n    {\n      \"id\": 2,\n      \"name\": \"Jane Smith\",\n      \"email\": \"jane.smith@example.com\",\n      \"phone\": \"+1-555-0102\",\n      \"status\": \"active\",\n      \"created_at\": \"2025-12-01 23:18:40\",\n      \"updated_at\": \"2025-12-01 23:1

In [17]:
# create_ticket
ticket_raw = send_mcp_message(
    "tools/call",
    {
        "name": "create_ticket",
        "arguments": {
            "customer_id": 1,
            "issue": "My order arrived damaged",
            "priority": "high"
        }
    },
    message_id=5
)

# get_customer_history
history_raw = send_mcp_message(
    "tools/call",
    {
        "name": "get_customer_history",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=6
)

if history_raw and "result" in history_raw:
    text_payload = history_raw["result"]["content"][0]["text"]
    history_data = json.loads(text_payload)
    print("\n Parsed get_customer_history result:")
    print(json.dumps(history_data, indent=2))


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:11] "POST /mcp HTTP/1.1" 200 -



[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 5,
  "method": "tools/call",
  "params": {
    "name": "create_ticket",
    "arguments": {
      "customer_id": 1,
      "issue": "My order arrived damaged",
      "priority": "high"
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP response:
{
  "jsonrpc": "2.0",
  "id": 5,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "{\n  \"success\": true,\n  \"ticket\": {\n    \"id\": 82,\n    \"customer_id\": 1,\n    \"issue\": \"My order arrived damaged\",\n    \"status\": \"open\",\n    \"priority\": \"high\",\n    \"created_at\": \"2025-12-01 23:40:11\"\n  }\n}"
      }
    ]
  }
}

[INFO] MCP request → tools/call
{
  "jsonrpc": "2.0",
  "id": 6,
  "method": "tools/call",
  "params": {
    "name": "get_customer_history",
    "arguments": {
      "customer_id": 1
    }
  }
}
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] MCP respon

## A2A

In [18]:
OPENROUTER_API_KEY = "hidden_for_privacy"

In [19]:
# ============================================================
# LangChain-based A2A
# Router Agent + CustomerDataAgent + SupportAgent
# Uses:
#   - OpenRouter Grok 4.1 free (via LangChain ChatOpenAI)
#   - existing MCPToolClient to talk to the DB
# ============================================================

import os
import json
from typing import Any, Dict, Optional, List

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
)

# ---------- 0. LLM via OpenRouter (Grok 4.1 free) ----------

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_OPENROUTER_MODEL = "x-ai/grok-4.1-fast:free"

# LangChain's ChatOpenAI expects the base URL ("/v1")
OPENROUTER_BASE_URL = OPENROUTER_URL.rsplit("/chat/completions", 1)[0]

llm = ChatOpenAI(
    base_url=OPENROUTER_BASE_URL,                 # "https://openrouter.ai/api/v1"
    api_key=OPENROUTER_API_KEY,
    model=DEFAULT_OPENROUTER_MODEL,
    temperature=0.0,
)

print("[INFO] LangChain LLM (OpenRouter Grok) initialized.")


# ---------- 1. Wrap your MCP tools as LangChain tools ----------

# Reuse your existing MCPToolClient and SERVER_URL from earlier cells
tools_client = MCPToolClient(SERVER_URL)


@tool
def lc_get_customer(customer_id: int) -> Dict[str, Any]:
    """Get a customer's full profile by ID from the MCP server."""
    return tools_client.call_tool("get_customer", {"customer_id": customer_id})


@tool
def lc_list_customers(status: str = "active", limit: int = 20) -> Dict[str, Any]:
    """List customers, optionally filtered by status."""
    return tools_client.call_tool(
        "list_customers",
        {"status": status, "limit": limit},
    )


@tool
def lc_get_customer_history(customer_id: int) -> Dict[str, Any]:
    """Get a customer's support / ticket history."""
    return tools_client.call_tool(
        "get_customer_history",
        {"customer_id": customer_id},
    )


@tool
def lc_create_ticket(
    customer_id: int,
    issue: str,
    priority: str = "medium",
) -> Dict[str, Any]:
    """Create a support ticket for a customer."""
    return tools_client.call_tool(
        "create_ticket",
        {
            "customer_id": customer_id,
            "issue": issue,
            "priority": priority,
        },
    )


@tool
def lc_update_customer(
    customer_id: int,
    name: Optional[str] = None,
    email: Optional[str] = None,
    phone: Optional[str] = None,
    status: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Update a customer's record. Only fields that are not None will be changed.
    """
    args: Dict[str, Any] = {"customer_id": customer_id}
    if name is not None:
        args["name"] = name
    if email is not None:
        args["email"] = email
    if phone is not None:
        args["phone"] = phone
    if status is not None:
        args["status"] = status

    return tools_client.call_tool("update_customer", args)


print("[INFO] MCP tools wrapped as LangChain tools.")


# ---------- 2. Generic helper: run an LLM agent with tools ----------

def run_llm_with_tools(
    user_input: str,
    tools: List[Any],
    system_prompt: str,
    payload: Optional[Dict[str, Any]] = None,
) -> Dict[str, Any]:
    """
    Core agent loop:
    - Adds system prompt + (optional) JSON payload context
    - Binds the tools to the LLM
    - Loops to handle tool calls until the model returns a final answer
    Returns:
        {
          "output": <final text>,
          "messages": [SystemMessage, HumanMessage, ToolMessage, ..., AIMessage],
        }
    """
    messages: List[Any] = [SystemMessage(content=system_prompt)]

    if payload:
        messages.append(
            HumanMessage(content=f"Context (JSON): {json.dumps(payload)}")
        )

    messages.append(HumanMessage(content=user_input))

    model = llm.bind_tools(tools)

    while True:
        ai_msg: AIMessage = model.invoke(messages)
        messages.append(ai_msg)

        tool_calls = getattr(ai_msg, "tool_calls", None)
        if not tool_calls:
            # No more tool calls -> final answer
            return {
                "output": ai_msg.content,
                "messages": messages,
            }

        # Handle tool calls one by one
        for call in tool_calls:
            tool_name = call["name"]
            tool_args = call.get("args", {})
            tool_id = call.get("id")

            # find the matching tool object
            tool_obj = next((t for t in tools if t.name == tool_name), None)
            if tool_obj is None:
                tool_result = {"error": f"Tool '{tool_name}' not found"}
            else:
                # LangChain tools can be called via .invoke(args)
                tool_result = tool_obj.invoke(tool_args)

            messages.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    tool_call_id=tool_id,
                )
            )


# ---------- 3. Define the specialized agents (Data + Support) ----------

# --- Customer Data Agent (read-only) ---

data_tools = [lc_get_customer, lc_list_customers, lc_get_customer_history]

DATA_SYSTEM_PROMPT = (
    "You are the Customer Data Agent.\n"
    "- You ONLY read customer data via the tools.\n"
    "- You NEVER promise support resolutions or create tickets.\n"
    "- You retrieve and summarize factual data about customers."
)


def run_customer_data_agent(
    user_input: str,
    payload: Optional[Dict[str, Any]] = None,
) -> Dict[str, Any]:
    return run_llm_with_tools(
        user_input=user_input,
        tools=data_tools,
        system_prompt=DATA_SYSTEM_PROMPT,
        payload=payload,
    )


# --- Support Agent (issues, tickets, updates) ---

support_tools = [
    lc_get_customer,
    lc_list_customers,
    lc_get_customer_history,
    lc_create_ticket,
    lc_update_customer,
]

SUPPORT_SYSTEM_PROMPT = (
    "You are the Support Agent.\n"
    "- You handle customer support issues and escalations.\n"
    "- You decide when to create tickets and with what priority.\n"
    "- You MUST use the tools for all customer data and updates.\n"
    "- When responding, explain clearly what you did."
)


def run_support_agent(
    user_input: str,
    payload: Optional[Dict[str, Any]] = None,
) -> Dict[str, Any]:
    return run_llm_with_tools(
        user_input=user_input,
        tools=support_tools,
        system_prompt=SUPPORT_SYSTEM_PROMPT,
        payload=payload,
    )


# ---------- 4. Router Agent (decides DATA vs SUPPORT) ----------

ROUTER_SYSTEM_PROMPT = (
    "You are the Router Agent.\n"
    "Classify the user's request into exactly one of two categories:\n"
    "- DATA: for customer data lookups / summaries only.\n"
    "- SUPPORT: for issues, complaints, changes, or ticket creation.\n\n"
    "Reply with ONLY the word DATA or SUPPORT."
)


def route_to_agent(user_input: str, payload: Optional[Dict[str, Any]] = None) -> str:
    """Use the LLM to decide whether to call Data or Support agent."""
    messages: List[Any] = [SystemMessage(content=ROUTER_SYSTEM_PROMPT)]
    if payload:
        messages.append(
            HumanMessage(content=f"Context (JSON): {json.dumps(payload)}")
        )
    messages.append(HumanMessage(content=user_input))

    resp: AIMessage = llm.invoke(messages)
    txt = (resp.content or "").upper()
    if "DATA" in txt and "SUPPORT" not in txt:
        return "DATA"
    if "SUPPORT" in txt and "DATA" not in txt:
        return "SUPPORT"
    # fallback heuristic
    if any(k in user_input.lower() for k in ["help", "issue", "problem", "ticket", "refund", "upgrade", "cancel"]):
        return "SUPPORT"
    return "DATA"


# ---------- 5. Multi-agent entry point (replaces old orchestrator.run) ----------

def run_multi_agent(
    user_text: str,
    payload: Optional[Dict[str, Any]] = None,
) -> Dict[str, Any]:
    """
    Main A2A entry point:
    - Router decides DATA vs SUPPORT
    - Then calls the corresponding agent (which uses tools via LangChain)
    """
    decision = route_to_agent(user_text, payload)
    print(f"🔀 Router decision: {decision}")

    if decision == "DATA":
        result = run_customer_data_agent(user_text, payload)
        result["agent"] = "CustomerDataAgent"
    else:
        result = run_support_agent(user_text, payload)
        result["agent"] = "SupportAgent"

    return result


print("[INFO] LangChain-based multi-agent A2A system initialized (Router + Data + Support).")

[INFO] LangChain LLM (OpenRouter Grok) initialized.
[INFO] MCP tools wrapped as LangChain tools.
[INFO] LangChain-based multi-agent A2A system initialized (Router + Data + Support).


## TEST Scenarios

### TEST 1 - Simple Query

In [20]:
print("=== Scenario: Simple Query – Get customer information for ID 5 ===")
res = run_multi_agent(
    "Get customer information for ID 5",
    payload={"customer_id": 5}
  )

print("Agent:", res["agent"])
print("Output:", res["output"])

=== Scenario: Simple Query – Get customer information for ID 5 ===
🔀 Router decision: DATA


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:19] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
Agent: CustomerDataAgent
Output: **Customer ID: 5**

- **Name**: Charlie Brown
- **Email**: new@email.com
- **Phone**: +1-555-0105
- **Status**: active
- **Created**: 2025-12-01 23:18:40
- **Updated**: 2025-12-01 23:34:16


### TEST 2 - Coordinated Query

In [21]:
print("\n=== Test 2: Upgrade Account ===")
res = run_multi_agent(
    "I'm customer 5 and need help upgrading my account.",
    payload={"customer_id": 5}
)

print("Agent:", res["agent"])
print("Output:", res["output"])


=== Test 2: Upgrade Account ===
🔀 Router decision: SUPPORT


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:39] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:40:50] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
Agent: SupportAgent
Output: Hello Charlie Brown (Customer ID: 5),

Thank you for reaching out about upgrading your account. I first retrieved your customer profile from our system, which confirms you're an active customer (email: new@email.com, phone: +1-555-0105).

I then checked your support ticket history, and I see you already have an **open ticket #55** specifically for "Request to upgrade account" (created 2025-12-01 23:38:22, priority: medium). No new ticket is needed at this time.

Could you provide more details on what kind of upgrade you're looking for (e.g., plan type, features needed)? I'll add notes to ticket #55 or escalate if required.

Let me know how else I can assist!


### TEST 3 - Complex Query

In [22]:
print("\n=== Test 3: Active customers with open tickets ===")
res = run_multi_agent(
    "Show me all active customers who have open tickets."
)

print("Agent:", res["agent"])
print("Output:", res["output"])


=== Test 3: Active customers with open tickets ===
🔀 Router decision: DATA


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:12] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HT

[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] S

INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:41:35] "POST /mcp HTTP/1.1" 200 -


[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
Agent: CustomerDataAgent
Output: ### Active Customers with Open Tickets

Here is a summary of all active customers who have at least one open ticket (based on their support/ticket history). I've included key profile details 

### TEST 4 - Escalation

In [23]:
print("\n=== Test 4: Urgent Refund ===")
res = run_multi_agent(
    "I've been charged twice. Please refund immediately!",
    payload={"customer_id": 1}
)

print("Agent:", res["agent"])
print("Output:", res["output"])


=== Test 4: Urgent Refund ===
🔀 Router decision: SUPPORT


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:42:10] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:42:10] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:42:16] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
Agent: SupportAgent
Output: Hello John Doe (ID: 1),

Thank you for contacting support. I'm sorry to hear about the double charge issue—that sounds frustrating.

**What I did:**
- Retrieved your customer profile (active status, email: john.new@example.com, phone: +1-555-7777).
- Reviewed your ticket history (you have 12 existing open/in-progress tickets, mostly related to login issues and a damaged order—no prior double-charge reports).
- Created a **new high-priority ticket (#83)** for "Charged twice for the same transaction - requesting immediate refund". It's now open and will be escalated to our billing team for urgent review.

To help us process this quickly, could you please reply with:
- The transaction ID(s) or order number(s) involved.
- The exact amounts and dates of the charges.
- A screenshot of your bank/statement if possible.

We'll investigate and aim to refund any duplicate charge ASAP (typically within 

### TEST 5 - Multi-Intent

In [24]:
print("\n=== Test 5: Update email + Ticket History ===")
res = run_multi_agent(
    "Update my email to new@email.com and show my ticket history.",
    payload={"customer_id": 5}
)

print("Agent:", res["agent"])
print("Output:", res["output"])


=== Test 5: Update email + Ticket History ===
🔀 Router decision: SUPPORT


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:42:33] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 23:42:33] "POST /mcp HTTP/1.1" 200 -


[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
[INFO] Received MCP message: tools/call
[INFO] Sending MCP response
Agent: SupportAgent
Output: Your email has been successfully updated to **new@email.com** in your customer profile (ID: 5). Here's your updated profile summary for confirmation:

- **Name**: Charlie Brown
- **Email**: new@email.com
- **Phone**: +1-555-0105
- **Status**: active

### Your Ticket History (7 tickets total):
1. **ID 63** (Open, Medium priority) - "Email notifications not being received" (Created: 2025-12-01 23:40:10)
2. **ID 72** (Resolved, Low priority) - "Typo in welcome email" (Created: 2025-12-01 23:40:10)
3. **ID 55** (Open, Medium priority) - "Request to upgrade account" (Created: 2025-12-01 23:38:22)
4. **ID 35** (Open, Medium priority) - "Email notifications not being received" (Created: 2025-12-01 23:37:51)
5. **ID 44** (Resolved, Low priority) - "Typo in welcome email" (Created: 2025-12-01 23:37:51)
6. **ID 8** (Open, Medium prior

## Conclusion

Through this project, I gained a deeper understanding of how multi-agent systems can coordinate using a shared tool interface and how an LLM can serve as the reasoning engine behind each agent’s behavior. I learned how to integrate LangChain’s tool-binding mechanism with an MCP server so agents can retrieve, update, and act on real database records. Implementing the Router Agent helped me understand intent classification and dynamic task allocation, while the CustomerDataAgent and SupportAgent showed me how specialized agents can collaborate through sequential tool calls to produce coherent end-to-end results.

Building the system also revealed several practical challenges. One challenge was aligning the LLM outputs with the exact tool names and JSON arguments expected by the MCP server, especially when LangChain and the MCP protocols had slightly different conventions. Another challenge was handling multi-step reasoning—ensuring agents could perform multiple tool calls, handle the responses, and then synthesize a final answer cleanly. Debugging tool calls, parsing tool responses, and managing the state across agents required careful design. Overall, the experience strengthened my understanding of agent coordination, tool execution pipelines, and how to integrate real backend systems into LLM-driven workflows.